In [22]:
%matplotlib inline
from tqdm import tqdm
from scipy.special import gammaln
import numpy as np
import pymc3 as pm
from theano import shared
import theano.tensor as tt
import theano
from pymc3.distributions.transforms import t_stick_breaking

In [3]:
import re
def plot_traces(trcs, varnames=None):
    '''Plot traces with overlaid means and values'''

    nrows = len(trcs.varnames)
    if varnames is not None:
        nrows = len(varnames)

    ax = pm.traceplot(trcs, varnames=varnames, figsize=(12,nrows*1.4),
                      lines={k: v['mean'] for k, v in
                             pm.summary(trcs,varnames=varnames).iterrows()})

    for i, mn in enumerate(pm.summary(trcs, varnames=varnames)['mean']):
        ax[i,0].annotate('{:.2f}'.format(mn), xy=(mn,0), xycoords='data',
                         xytext=(5,10), textcoords='offset points', rotation=90,
                         va='bottom', fontsize='large', color='#AA0022')
def strip_derived_rvs(rvs):
    '''Remove PyMC3-generated RVs from a list'''

    ret_rvs = []
    for rv in rvs:
        if not (re.search('_log',rv.name) or re.search('_interval',rv.name)):
            ret_rvs.append(rv)
    return ret_rvs

In [4]:
import pickle
with open('data/gch-100000-10-0.25-5.00.pkl', 'rb') as f:
  GCH, TH = pickle.load(f)
S = 10
npdata = np.array(GCH, dtype = int)
V = len(GCH)
V, type(V)

(12276, int)

In [5]:
def get_vmask():
    v_mask = np.zeros([S,S,S])
    for s in np.arange(S):
        for q in np.arange(s,S):
            c1 = np.zeros(s)
            c2 = np.ones(q-s)
            c3 = np.zeros(S-q)
            v_mask[s,q]= np.concatenate((c1, c2, c3))
    return tt.constant(v_mask)

In [6]:
def get_imask():
    s_mask = np.zeros([V,S])
    q_mask = np.zeros([V,S])
    for w in np.arange(V):
        f, l = npdata[w, 0] , npdata[w, 1]
        s_mask[w] = np.concatenate((np.ones(f+1), np.zeros(S - f -1)))
        q_mask[w] = np.concatenate((np.zeros(l), np.ones(S-l)))
    i_mask = np.einsum("is,iq->isq", s_mask, q_mask)
    return tt.constant(i_mask)

In [76]:
def get_Risq(a,b):
    
    #data preparation
    count_capture = npdata[:, 2]
    n = np.broadcast_to(np.triu(np.ones([S,S]), 0).cumsum(axis =1), (V, S, S))
    capture = count_capture[:, np.newaxis, np.newaxis] *  np.ones((V, S, S))
    missed = np.triu(n - capture, 0)
    missed = np.clip(missed,a_min=0,a_max=S)
    
    n = tt.constant(n)
    capture = tt.constant(capture)
    missed = tt.constant(missed)
    
    # The beta binomial
    R_isq =   tt.gammaln(n+1)
    R_isq -=  tt.gammaln(capture+1)
    R_isq -=  tt.gammaln(missed+1)   
    R_isq +=  tt.gammaln(capture+a)
    R_isq +=  tt.gammaln(missed+b)
    R_isq -=  tt.gammaln(n+a+b)
    R_isq +=  tt.gammaln(a+b) - tt.gammaln(a) - tt.gammaln(b)

    print(n.eval()[0])
    print(capture.eval()[0])
    print(missed.eval()[0])
    # The above is the computation of the log, so we take the exponent
    return tt.exp(R_isq)

In [77]:
def get_Risq0(a,b):
    # n = q - s + 1
    n = tt.constant(np.triu(np.ones([S,S]), 0).cumsum(axis =1))
    R0_sq = tt.gammaln(n + b) - tt.gammaln(n + a + b) + tt.gammaln(a + b) - tt.gammaln(b)
    return tt.exp(R0_sq)

In [100]:
def logp_capture(arr, phi, a, b, U):
    def ll_capture_f(mycaptures):
        
        # Masks: try to compute these once.
        phi_mask = get_vmask()  # S x S x S   all possible arrivals and departures for the "SURVIVAL"
        i_mask   = get_imask()  # V x S x S   matrix with real possible arrival and departures
        R_isq = get_Risq(a, b)  # V x S x S   capture probability for observed
        R0_sq = get_Risq0(a, b) # S x S       capture probability for non-observed
        
        # Likelihood of Capture Li
        phi_v = tt.pow(1-phi, phi_mask)
        phi_v = tt.prod(phi_v, axis=2)
        LD = arr[:, np.newaxis] * phi_v * phi
        LD_isq = tt.mul(LD, i_mask)
        Li = tt.batched_tensordot(LD_isq, R_isq, axes = 2)
        print((LD_isq.eval()[0] * R_isq.eval()[0]).sum())
        
        # Likelihood of No-Capture L0
        v0_mask = 1 - phi_mask[0] # tihs is reuse
        LD0 = tt.mul(LD, v0_mask)
        L0 = tt.sum(LD0 * R0_sq)
        
        # Multinomial
        obj1 = tt.gammaln(tt.constant(V)+U+1.0) - tt.gammaln(U+1.0) - tt.gammaln(tt.constant(V)+ 1.0) #
        obj2 = tt.sum(tt.log(Li))
        obj3 = U * tt.log(L0)
        
        objective = obj1 + obj2 + obj3 
        
        return objective
    
    return ll_capture_f

In [101]:
birth_init, dep = (np.array([0.4       , 0.06666667, 0.06666667, 0.06666667, 0.06666667,
          0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667]),
 np.array([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 1. ]))
#for UU in range(87600, 88400, 10):
#    print("Total Unseen:", UU, "/ Likelihood:", logp_capture(birth_init, dep, .25,  5., UU)(npdata).eval().round(4))
print("Total Unseen:", 10, "/ Likelihood:", logp_capture(birth_init, dep, 1, 1 , 10)(npdata).eval().round(4))

[[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [ 0.  0.  1.  2.  3.  4.  5.  6.  7.  8.]
 [ 0.  0.  0.  1.  2.  3.  4.  5.  6.  7.]
 [ 0.  0.  0.  0.  1.  2.  3.  4.  5.  6.]
 [ 0.  0.  0.  0.  0.  1.  2.  3.  4.  5.]
 [ 0.  0.  0.  0.  0.  0.  1.  2.  3.  4.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  2.  3.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  2.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 0. 0. 1. 2. 3. 4. 5. 6. 7.]
 [0. 0. 0. 0. 1. 2. 3. 4. 5. 6.]
 [0. 0. 0. 0. 0. 1. 2. 3. 4. 5.]
 [0. 0. 0. 0. 0. 0. 1. 2. 3. 4.]
 [0. 0. 0. 0. 0. 0. 0. 1. 2. 3.]
 [0. 0.

In [69]:
npdata[:, 0], npdata[:, 1] 

(array([2, 7, 0, ..., 0, 8, 2]), array([2, 7, 1, ..., 0, 9, 6]))

In [47]:
# custom log-liklihood
# model
with pm.Model() as model2:
    # parameters
    U = pm.Uniform('Unseen', lower=V, upper=100*V)
    a = pm.Uniform('alpha', .01, .1)
    b = pm.Uniform('beta', 1., 10.)
    phi    = pm.Uniform('departure', 0.1, 1., shape=S)
    arr    = pm.Dirichlet('arrival', a=np.array([1./S]*S), shape=S)
    
    # Model Log-likelihood
    ob = pm.DensityDist('x', logp_capture(arr, phi, a, b, U), observed=npdata)

TypeError: 'TensorVariable' object does not support item assignment

In [ ]:
with model:
  #posterior = pm.sample(3, tune=0, chains=1, progressbar=False)
  step = pm.NUTS()
  #posterior = pm.sample(100000, step=step, njobs = 4)
  posterior = pm.sample(1000, cores = 1, step = step)

In [ ]:
for RV in model.basic_RVs:
    print(RV.name, RV.logp(model.test_point))

In [ ]:
pm.energyplot(posterior);

In [ ]:
plot_traces(posterior[2000:], varnames=['Unseen', 'alpha', 'beta']);

In [ ]:
pm.summary(posterior).round(2)

In [ ]:
V

In [ ]:
model['Unseen'].logp(model.test_point)

In [ ]:
model['a'].logp(model.test_point)